NESTED QUERIES
------------

In [1]:
%load_ext sql
%sql sqlite:///world-db

Recall the following nested query from the lecture:

In [10]:
%%sql
SELECT C.Name 
FROM Country C 
WHERE C.code =
 (SELECT C.CountryCode
  FROM City C 
  Where C.name = 'Berlin');

 * sqlite:///world-db
Done.


Name
Germany


**Exercise #1:** Can we do this query without nesting?  Write your query here:

In [12]:
%%sql
SELECT T.Name 
FROM Country T join City C on T.code = C.CountryCode
Where C.name = 'Berlin'

 * sqlite:///world-db
Done.


Name
Germany


The next query finds all countries in Europe with population that exceeds the average population of a country in Europe.

In [13]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
 AND C.Population > (
    SELECT AVG(Population)
    FROM Country
    WHERE Continent = 'Europe') ;

 * sqlite:///world-db
Done.


Name
Germany
Spain
France
United Kingdom
Italy
Poland
Romania
Russian Federation
Ukraine


The next query finds all countries in Europe with population more than 50 million. Notice here that we can nest a SQL query in the `FROM` clause.

In [17]:
%%sql
SELECT C.Name
FROM (SELECT Name, Continent
      FROM Country
      WHERE Population >50000000) AS C 
WHERE C.Continent = 'Europe' ;

 * sqlite:///world-db
Done.


Name
Germany
France
United Kingdom
Italy
Russian Federation
Ukraine


The `WITH` clause can be added before the `SELECT` statement to define a table that can be used in the main query statement. This can be an alternative syntax to nesting a statement in the `FROM` clause.

In [18]:
%%sql
WITH C AS (SELECT Name, Continent
      FROM Country
      WHERE Population >50000000)
SELECT C.Name
FROM C
WHERE C.Continent = 'Europe' ;

 * sqlite:///world-db
Done.


Name
Germany
France
United Kingdom
Italy
Russian Federation
Ukraine


It is possible to have multiple `WITH` clauses in the beginning of a query, separated by commas.

**Exercise #2** Can you unnest this query?

In [26]:
%%sql SELECT Name
FROM Country 
WHERE Population > 50000000
And Continent = 'Europe'

 * sqlite:///world-db
Done.


Name
Germany
France
United Kingdom
Italy
Russian Federation
Ukraine


The next query outputs all countries in Europe that have some city with population more than 5 million. It uses the `IN` operator.

In [27]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
AND C.Code IN (SELECT CountryCode
               FROM City
               WHERE Population > 5000000);

 * sqlite:///world-db
Done.


Name
United Kingdom
Russian Federation


An equivalent way to write the above query is by using the `EXISTS` statement. 

We call the subquery below `correlated`, because it uses a variable name that is `not` defined in the scope of the subquery.

In [28]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
AND EXISTS (SELECT *
            FROM City T
            WHERE T.Population > 5000000 
            AND T.CountryCode = C.Code);

 * sqlite:///world-db
Done.


Name
United Kingdom
Russian Federation


We can even rewrite the above query using `ANY`. Unfortunately, SQLite does not support ANY/ALL, and thus running the query below will give an error.

In [29]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
AND 5000000 <= ANY (SELECT T.Population
                    FROM City T
                    WHERE T.CountryCode = C.Code);

 * sqlite:///world-db
(sqlite3.OperationalError) near "SELECT": syntax error
[SQL: SELECT C.Name FROM Country C
WHERE C.Continent = 'Europe'
AND 5000000 <= ANY (SELECT T.Population
                    FROM City T
                    WHERE T.CountryCode = C.Code);]
(Background on this error at: http://sqlalche.me/e/e3q8)


**Exercise #3** Can you unnest the above query?

In [37]:
%%sql
SELECT C.Name
FROM Country C join City T on T.CountryCode = C.Code
WHERE C.Continent = 'Europe'
AND 5000000 < T.Population;

 * sqlite:///world-db
Done.


Name
United Kingdom
Russian Federation


Our final query for this activity outputs all countries in Europe that have **all** cities with population less than 1 million. 

In [32]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
AND NOT EXISTS (SELECT *
                FROM City T
                WHERE T.Population > 1000000 
                AND T.CountryCode = C.Code);

 * sqlite:///world-db
Done.


Name
Albania
Andorra
Belgium
Bosnia and Herzegovina
Switzerland
Denmark
Estonia
Finland
Faroe Islands
Gibraltar


**Exercise #4** Can you write the above query without nesting? Use instead the `EXCEPT` set operator!

In [36]:
%%sql
SELECT C.Name
FROM Country C join City T on T.CountryCode = C.Code
WHERE C.Continent = 'Europe'
Group by C.Name
Having Max(T.Population)<1000000;

 * sqlite:///world-db
Done.


Name
Albania
Andorra
Belgium
Bosnia and Herzegovina
Croatia
Denmark
Estonia
Faroe Islands
Finland
Gibraltar


In [39]:
%%sql
SELECT C.Name
FROM Country C 
WHERE C.Continent = 'Europe'
EXCEPT
SELECT C.Name
FROM Country C join City T on T.CountryCode = C.Code
WHERE C.Continent = 'Europe'
And T.Population>1000000;

 * sqlite:///world-db
Done.


Name
Albania
Andorra
Belgium
Bosnia and Herzegovina
Croatia
Denmark
Estonia
Faroe Islands
Finland
Gibraltar
